#This RAG framework for using DRAGON Embedding model with Reranking, and uses Ollama-Mistral model for generating answers through Llamaindex

In [ ]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q torch sentence-transformers
!pip install -q -U pymilvus
!pip install -q datasets
!pip install -q transformers
!pip install -q spacy
!pip install -q tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 

In [ ]:
import pandas as pd
csv_path = "/content/halubench_halueval_pass.csv"
data = pd.read_csv(csv_path)
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in data.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")
# Extract 'contexts' and 'questions'
text_data = data['contexts'].tolist()
questions_data = data['question'].tolist()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

def split_text_by_length(text, max_length=4096):
    """
    Split the input text into chunks of max_length, ensuring no chunk exceeds max_length.
    """
    split_texts = []
    for i in range(0, len(text), max_length):
        chunk = text[i:i + max_length]
        if len(chunk) <= max_length:
            split_texts.append(chunk)
    return split_texts

split_texts = []
for text in text_data:
    split_texts.extend(split_text_by_length(text, max_length=4096))
#Initialize a list to store the mapping between questions and their context chunks
mapped_questions = []

# Map each question to its corresponding context chunks
for idx, text in enumerate(text_data):
    chunks = split_text_by_length(text, max_length=4096)  # Split context into chunks
    for _ in chunks:  # For each chunk of context, repeat the question
        mapped_questions.append(questions_data[idx])  # Map the question to each chunk

# Now both mapped_questions and split_texts should have the same length
assert len(mapped_questions) == len(split_texts), "Mapped questions and split texts are not aligned."

# Display the split chunks
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i+1}:")
    print(chunk)
    print()

Streaming output truncated to the last 5000 lines.
Hemi-Sync is a trademarked brand name for a patented process used to create audio patterns containing binaural beats, which are commercialized in the form of audio CDs.Robert Allan Monroe, also known as Bob Monroe (October 30, 1915 â€“ March 17, 1995), was a radio broadcasting executive who became known for his research into altered consciousness and founding The Monroe Institute.

Chunk 3325:
RKO 281 is a 1999 American historical drama film directed by Benjamin Ross and starring Liev Schreiber, James Cromwell, Melanie Griffith, John Malkovich, Roy Scheider and Liam Cunningham. The film depicts the troubled production behind the 1941 film "Citizen Kane". He became known during the late 1990s and early 2000s, having appeared in several independent films, and later mainstream Hollywood films, including the "Scream" trilogy of horror films, "Ransom" (1996), "Phantoms" (1998), "The Sum of All Fears" (2002), "The Omen" (2006), "" (2009), "T

#Retrieval

In [ ]:
from pymilvus import MilvusClient, utility

client = MilvusClient("milvus.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 94f5ebbb1cd541a3aa79c446b9ca366a


In [ ]:
from pymilvus import (
    connections, FieldSchema, CollectionSchema, DataType, Collection, utility
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

# Connect to Milvus using the cluster information from Zilliz Cloud
connections.connect(
    alias="default",
    uri="https://in03-7a0bd7c7d07bbe2.serverless.gcp-us-west1.cloud.zilliz.com",
    token="2fd0d35c5daca9322a69814ef89023098b12df8a1e716334ac601149f386dbd4fd7b2ad5265df24b6a7452ca1ddd097c1fb572f8"
)

# Optionally, check the connection status
connected = connections.has_connection(alias="default")
print(f"Connection established: {connected}")

Connection established: True


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('facebook/dragon-plus-query-encoder')
query_encoder = AutoModel.from_pretrained('facebook/dragon-plus-query-encoder')
context_encoder = AutoModel.from_pretrained('facebook/dragon-plus-context-encoder')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Define collection schema
dense_dim = 768  # Dimension of embeddings
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=675),  # Field for question
    FieldSchema(name="context", dtype=DataType.VARCHAR, max_length=8192),  # Field for context
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),  # Dense vector field
]
schema = CollectionSchema(fields)

In [ ]:
# Create or Load Collection
col_name = "rag_search"
if utility.has_collection(col_name):
    Collection(col_name).drop()  # Dropping existing collection (optional)
col = Collection(col_name, schema, consistency_level="Strong")

In [ ]:
# Initialize lists to store data for bulk insertion
questions_to_insert = []
contexts_to_insert = []
embeddings_to_insert = []
# Insert Data into Milvus
for idx, (question, context) in enumerate(zip(mapped_questions, split_texts)):
    # Tokenize and encode the context
    ctx_input = tokenizer(context, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Generate dense embeddings for the context (contextual embedding)
    ctx_emb = context_encoder(**ctx_input).last_hidden_state[:, 0, :].detach().numpy()

    # Flatten the numpy array to a list of floats
    ctx_emb = ctx_emb.flatten().tolist()

    # Append each field separately for insertion
    questions_to_insert.append(question)  # Append the question
    contexts_to_insert.append(context)    # Append the context
    embeddings_to_insert.append(ctx_emb)  # Append the dense vector

# Prepare the data in the correct format for Milvus insertion
data_to_insert = [questions_to_insert, contexts_to_insert, embeddings_to_insert]

In [ ]:
# Create Indices for Dense Vectors
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}  # Dense vector index
col.create_index("dense_vector", dense_index)

Status(code=0, message=)

In [ ]:
# Insert data into the collection
col.insert(data_to_insert)
# Load Collection into Memory for Efficient Search
col.load()

print(f"Inserted {len(questions_to_insert)} documents into the Milvus collection '{col_name}'.")

Inserted 4990 documents into the Milvus collection 'rag_search'.


#Reranking

In [ ]:
from sentence_transformers import CrossEncoder

# Initialize the cross-encoder
device = "cuda" if torch.cuda.is_available() else "cpu"
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#Generation

In [ ]:
!pip install llama-index-llms-ollama

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏ 119 KB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕▏  40 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  63 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  89 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 108 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 117 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 142 MB/4.1 GB                  pulling manifest 
pullin

In [ ]:
from llama_index.llms.ollama import Ollama
# Initialize the Ollama LLM
llm = Ollama(model="mistral", request_timeout=300.0, temperature=0.1)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import (Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate)

# Define the prompt template for generating the answer
prompt_template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    """

In [ ]:
questions_data = data['question'].tolist()
contexts_data = data['contexts'].tolist()

In [ ]:
# Tokenize and generate embeddings for all questions
question_embeddings = []
for question in questions_data:
    query_input = tokenizer(question, return_tensors='pt', truncation=True, padding=True)
    query_emb = query_encoder(**query_input).last_hidden_state[:, 0, :].detach().numpy()

    # Convert numpy array to a list of floats
    query_emb = query_emb.flatten().tolist()  # Ensure this is a list of floats

    question_embeddings.append(query_emb)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from datasets import load_dataset

from pymilvus import (
    AnnSearchRequest,
     WeightedRanker
)

# Function to perform dense vector search and retrieve the top-k results
def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "IP", "params": {"nprobe": 10}}  # Use IP distance metric
    results = col.search(
        [query_dense_embedding],  # Pass the dense embedding
        anns_field="dense_vector",  # Field containing dense vectors in Milvus
        limit=limit,  # Number of results to retrieve
        output_fields=["context", "question"],  # Retrieve both 'context' and 'question'
        param=search_params,
    )[0]  # Access the first hit

    # Extract the 'question' and 'context' from each hit
    return [(hit.entity.get("question"), hit.entity.get("context")) for hit in results]

In [ ]:
def rerank_contexts(question, retrieved_results):
    """
    Rerank the retrieved contexts using a cross-encoder model.

    Args:
        question (str): The input question.
        retrieved_results (list): List of tuples (question, context) from dense search.

    Returns:
        str: The best context after reranking.
    """
    # Prepare inputs for the cross-encoder: pairs of (question, context)
    rerank_inputs = [(question, context) for _, context in retrieved_results]

    # Compute relevance scores for each (question, context) pair
    rerank_scores = cross_encoder.predict(rerank_inputs)

    # Find the index of the best context with the highest score
    best_index = rerank_scores.argmax()

    # Return the best context after reranking
    return rerank_inputs[best_index][1]  # Return the best context

In [ ]:
# Initialize lists to store results
generated_answers = []
retrieved_contexts = []
# Loop through each question and generate an answer based on retrieved context
for i, question_emb in enumerate(question_embeddings):
    # Perform dense vector search and retrieve the top-k results
    retrieved_results = dense_search(col, question_emb, limit=5)  # Get top 5 contexts

    # Rerank the retrieved contexts using the cross-encoder
    best_context = rerank_contexts(questions_data[i], retrieved_results)

    # Generate the answer using the Mistral model with the reranked best context and question
    prompt = f"Question: {questions_data[i]}\nContext: {best_context}\nAnswer:"
    response = llm.complete(prompt)

    # Append the generated answer and reranked context to the respective lists
    generated_answers.append(response)
    retrieved_contexts.append(best_context)

    # Print the generated answer for each question
    print(f"Generated Answer for question '{questions_data[i]}':\n{response}")
    print('-' * 50)

Streaming output truncated to the last 5000 lines.
--------------------------------------------------
Generated Answer for question 'The fair that features organic food in Unity, Maine takes place in which county? ':
 The fair takes place in Knox County, Maine, not Lake County, Colorado. This is because the context provided refers to a location in Maine (Unity) and there's no mention of any connection to Colorado or Lake County.
--------------------------------------------------
Generated Answer for question 'What co-educational college-preparatory school is located in the same state that Mary Richardson Walker's parents were from?':
 The state that Mary Richardson Walker's parents were from (and where Adelaide Miethke was also from) is South Australia. However, the question does not specify which co-educational college-preparatory school in South Australia is relevant to this context. The School of the Air is a distance education program, not a specific school.
-----------------------

In [ ]:
questions = data['question'].tolist() #1
contexts = data['contexts'].tolist()
ground_truths = data['ground_truth'].tolist()
# Combine the result into a DataFrame
output_df = pd.DataFrame({
    'question': data['question'],
    'contexts': data['contexts'],
    'answer': generated_answers,    # Generated answers for each question
    'ground_truth': data['ground_truth']   # The ground truth from the original dataset
})
data['answer'] = generated_answers
data['retrieved_documents'] = retrieved_contexts

In [ ]:
output_csv_path = 'halueval_RAG_Advanced.csv'
data.to_csv(output_csv_path, index=False)